In [2]:
search_for = 137
start_from = 0
threads = 10
thread = 6
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.002916097640991211
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 5
all cases: 354865621
type: [1, 1, 1, 1, 137] 137
cases of this type: 352275361
100000 17.371293330021942
200000 32.453222556754824
300000 41.486524095870934
400000 33.44993477139653
500000 39.93999307947945
600000 42.10641947211443
700000 36.476487934724524
800000 40.19670065091629
900000 38.97229227152373
1000000 34.27205524003684
1100000 34.487280584995744
1200000 32.753730255810844
1300000 18.26471877120958
1400000 24.12084282286514
1500000 34.786526583575736
1600000 32.3352241297092
1700000 34.57658190718146
1800000 32.24503086315626
1900000 35.05993163452834
2000000 37.00793352324325
2100000 32.97137721214265
2200000 34.88916805490381
2300000 32.487518694576984
2400000 35.32807962368237
2500000 34.5645006507009
2600000 13.97247668694874
2700000 27.91772299651376
2800000 43.46833458611081
2900000 43.76444600986106
3000000 40.61577036629

29800000 31.398863261045044
29900000 32.40058857586085
30000000 32.6191644233353
30100000 33.163662480423056
30200000 32.43596789863705
30300000 32.92035073475226
30400000 34.917556261233095
30500000 33.63658752558155
30600000 32.341359421557655
30700000 13.445697859222
30800000 35.326594346795495
30900000 34.91030989690537
31000000 33.89346706424675
31100000 20.836989518146414
31200000 24.953027185875545
31300000 34.44458253818714
31400000 34.58836402361904
31500000 34.80419809613702
31600000 33.35460025044169
31700000 34.59391393301691
31800000 34.05600732231811
31900000 32.77790457620587
32000000 32.7752225365019
32100000 31.689308009664995
32200000 7.5434538237615065
32300000 32.86564748193443
32400000 32.03060199161547
32500000 32.79225791209028
32600000 33.2643967261584
32700000 33.117044735245194
32800000 32.710714695971085
32900000 33.67639613483958
33000000 33.7747671835862
33100000 32.891347873475816
33200000 21.29265872600917
33300000 25.29960772217655
33400000 34.2249011816

59700000 24.222742488134315
59800000 30.195000992990757
59900000 28.770436544917818
60000000 28.947436546739688
60100000 27.85744700310291
60200000 27.561703638944756
60300000 27.245739925438237
60400000 23.000735640670435
60500000 11.649726861408269
60600000 26.94143553637749
60700000 27.491399695629518
60800000 27.72096319629709
60900000 28.41626626019391
61000000 28.674610455774875
61100000 29.005710079078813
61200000 28.68707158737191
61300000 14.144354794154614
61400000 27.58371319848096
61500000 30.17907731213474
61600000 29.679570950402166
61700000 31.17430476117888
61800000 30.606474897519355
61900000 30.105452130681336
62000000 31.34824877991153
62100000 31.357110545362914
62200000 13.756989519455738
62300000 26.470169734857205
62400000 29.64177328028147
62500000 29.33919555047319
62600000 29.95099321491158
62700000 28.994681002274696
62800000 27.578260116756606
62900000 27.526219578827618
63000000 16.57276218892614
63100000 19.2194855515532
63200000 27.442960097949122
6330000

89400000 8.663381409692336
89500000 27.495566341075556
89600000 26.222877605402488
89700000 26.83436952350256
89800000 27.107840937869057
89900000 26.542925090688975
90000000 28.33325230891319
90100000 26.30197471328685
90200000 26.80436824629833
90300000 27.310354108986864
90400000 26.54460874349901
90500000 26.130678558473242
90600000 26.416034525226113
90700000 9.779218622467893
90800000 22.411051115737617
90900000 23.909586340014947
91000000 22.614133359101423
91100000 23.53437559832822
91200000 23.520385352765647
91300000 11.097863158250451
91400000 18.242697887778544
91500000 23.44635065061057
91600000 23.076782137630236
91700000 23.177142263054705
91800000 23.456044728216067
91900000 16.614871588305082
92000000 17.92620858028836
92100000 25.057070916552238
92200000 25.052410321753868
92300000 26.290802846783624
92400000 25.672862222726696
92500000 26.339004188684587
92600000 25.294296618802868
92700000 26.449400399575513
92800000 26.616468150484092
92900000 26.801228665892346
93

118500000 23.742891369406195
118600000 23.779169922700948
118700000 23.907870104616013
118800000 22.27412825793127
118900000 22.09129387789664
119000000 21.597006614473816
119100000 21.76550109873423
119200000 6.66701418376764
119300000 21.807459218813026
119400000 21.891864046556808
119500000 21.202401425138394
119600000 7.229582698008069
119700000 19.965067287540343
119800000 21.215064920912212
119900000 21.53855758468362
120000000 11.54660017098351
120100000 18.21577268530828
120200000 21.310173147572744
120300000 21.095809051821252
120400000 21.79209321324508
120500000 22.455234643916604
120600000 22.61430310449203
120700000 23.43118724072923
120800000 23.124123949358097
120900000 23.297689876194696
121000000 23.394459743169044
121100000 23.10978107906086
121200000 22.734819905649967
121300000 22.850547048746925
121400000 21.40479180716461
121500000 21.152460613075252
121600000 20.802911861895513
121700000 21.43400168102901
121800000 6.519476903753889
121900000 20.52767097674065
12

...

233300000 11.407105159401196
233400000 11.26343459022875
233500000 11.8580834572953
233600000 11.633896549870965
233700000 12.127287542394441
233800000 12.42758748317035
233900000 12.538512217115173
234000000 12.580438845881982
234100000 12.145296115224252
234200000 14.122953416135312
234300000 12.241254534936092
234400000 11.975790166902309
234500000 11.326219335344497
234600000 11.280918079561626
234700000 10.828495516996933
234800000 10.701787969653894
234900000 3.83665722791437
235000000 10.829724309321362
235100000 10.615430765613352
235200000 10.468782584160788
235300000 4.523326579686361
235400000 8.901490090909865
235500000 10.96810042127871
235600000 10.767098102420434
235700000 8.41517685361225
235800000 6.829818907421116
235900000 10.939256631852057
236000000 11.222378203498389
236100000 11.648360161142053
236200000 11.567242803535436
236300000 11.883993214166656
236400000 12.42527199250951
236500000 12.058940137805557
236600000 11.989626950660126
236700000 12.12783062823389

262200000 9.662600609304716
262300000 8.771932050553387
262400000 9.197172476081848
262500000 9.129173254568473
262600000 9.289944349164704
262700000 9.04844430288081
262800000 8.838890792195587
262900000 7.705921648510093
263000000 4.485235309647148
263100000 8.289678145180789
263200000 8.46318241074468
263300000 8.184284810798372
263400000 8.360777899359384
263500000 8.577540376978899
263600000 2.4110130284331155
263700000 8.091795295451663
263800000 8.594373563919067
263900000 8.478469106567438
264000000 8.439174624893756
264100000 8.546257263056974
264200000 5.721898084796703
264300000 5.842896092464543
264400000 9.064458476574321
264500000 9.050777507077559
264600000 9.238651147381912
264700000 9.327471035551316
264800000 9.447970355973078
264900000 9.675504136306401
265000000 9.330946606167513
265100000 9.269603934743369
265200000 9.689089064325849
265300000 9.171967098003565
265400000 8.689115681389788
265500000 4.455797793509675
265600000 7.172885635911614
265700000 8.214010265

291500000 6.241417154811568
291600000 6.039604061196054
291700000 6.134160200302772
291800000 6.273489088592101
291900000 1.123994069001954
292000000 5.9496066597017165
292100000 6.261838122436291
292200000 6.148096205427003
292300000 5.898078838161497
292400000 6.509862764104668
292500000 6.093977125637309
292600000 6.531544993806944
292700000 3.792694345345843
292800000 4.981297871432026
292900000 6.319852289058673
293000000 6.673377298445638
293100000 6.815209040619617
293200000 6.279224265495923
293300000 6.643678264730483
293400000 6.612798435642609
293500000 6.21329998077945
293600000 2.1131866176833918
293700000 6.910429448039457
293800000 6.500296319823981
293900000 5.82918013944607
294000000 6.372910817575361
294100000 5.8162611906948705
294200000 5.793353443393231
294300000 5.872017326890094
294400000 4.541282691234362
294500000 2.965296770124472
294600000 5.687577026012079
294700000 5.7989333711967825
294800000 5.834343822490735
294900000 6.555749675870464
295000000 6.171534

320600000 3.588561515290653
320700000 3.7200362622283847
320800000 3.467030579407913
320900000 3.6473645004405326
321000000 3.6470097140503004
321100000 3.548142235509368
321200000 1.9498079874821583
321300000 2.790665304429542
321400000 3.6970146696514425
321500000 3.4210229267407586
321600000 3.209875458377465
321700000 1.4712520058295846
321800000 3.4252794062620784
321900000 3.3499159277901054
322000000 3.4928381496480645
322100000 3.489665724494796
322200000 3.1793563145672263
322300000 3.3138298856522654
322400000 3.2604967251107904
322500000 3.1797722943742115
322600000 3.075417071074194
322700000 2.0503387360633267
322800000 1.9052713559360115
322900000 3.013029554414642
323000000 3.1595456696344963
323100000 3.136120670075786
323200000 3.418526731962583
323300000 3.3217770125127246
323400000 3.157253413251645
323500000 3.4873603457221627
323600000 3.2943849254238735
323700000 3.3681761352586106
323800000 1.5342904731977463
323900000 3.045245754179458
324000000 3.26610006061736

349200000 0.49498811957898536
349300000 0.47831299298858043
349400000 0.569050336174365
349500000 0.32950487823530306
349600000 0.39428111455492976
349700000 0.16959318019887343
349800000 0.1421604535297175
349900000 0.29750832318811615
350000000 0.3347884325277951
350100000 0.24781352327570716
350200000 0.30314631784856716
350300000 0.26063136306815615
350400000 0.32691352866954737
350500000 0.2736308679355562
350600000 0.21730814858355058
350700000 0.261008920553518
350800000 0.22706650469863413
350900000 0.3306627438548883
351000000 0.10080053585000634
351100000 0.14906753249857427
351200000 0.21806538947101567
351300000 0.16299772465709447
351400000 0.16813077654582262
351500000 0.22083452535167006
351600000 0.1864298962527487
351700000 0.156392968487374
351800000 0.144079883454895
351900000 0.13778385942056112
352000000 0.14387398012257815
352100000 0.1769600328217235
352200000 0.1334048007439183
best so far: 0
type: [1, 1, 1, 137, 1] 137
cases of this type: 2571353
352300000 0.04